In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import wordnet as wn 
import csv

In [2]:
df_tr=pd.read_csv('Training.csv')

In [3]:
df_tt=pd.read_csv('Testing.csv')

In [4]:
df_tr.head()

itching  skin_rash  nodal_skin_eruptions  continuous_sneezing  shivering  \
0        1          1                     1                    0          0   
1        0          1                     1                    0          0   
2        1          0                     1                    0          0   
3        1          1                     0                    0          0   
4        1          1                     1                    0          0   

   chills  joint_pain  stomach_pain  acidity  ulcers_on_tongue  ...  \
0       0           0             0        0                 0  ...   
1       0           0             0        0                 0  ...   
2       0           0             0        0                 0  ...   
3       0           0             0        0                 0  ...   
4       0           0             0        0                 0  ...   

   blackheads  scurring  skin_peeling  silver_like_dusting  \
0           0         0             0                    0   
1           0         0             0                    0   
2           0         0             0                    0   
3           0         0             0                    0   
4           0         0             0                    0   

   small_dents_in_nails  inflammatory_nails  blister  red_sore_around_nose  \
0                     0                   0        0                     0   
1                     0                   0        0                     0   
2                     0                   0        0                     0   
3                     0                   0        0                     0   
4                     0                   0        0                     0   

   yellow_crust_ooze         prognosis  
0                  0  Fungal infection  
1                  0  Fungal infection  
2                  0  Fungal infection  
3                  0  Fungal infection  
4                  0  Fungal infection  

[5 rows x 133 columns]

In [5]:
symp=[]
disease=[]
for i in range(len(df_tr)):
    symp.append(df_tr.columns[df_tr.iloc[i]==1].to_list())
    disease.append(df_tr.iloc[i,-1])

In [6]:
symp[0]

['itching', 'skin_rash', 'nodal_skin_eruptions', 'dischromic _patches']

In [7]:
disease[0]

'Fungal infection'

# I- GET ALL SYMPTOMS

In [8]:
all_symp_col=list(df_tr.columns[:-1])
def clean_symp(sym):
    return sym.replace('_',' ').replace('.1','').replace('(typhos)','').replace('yellowish','yellow').replace('yellowing','yellow') 

In [9]:
all_symp=[clean_symp(sym) for sym in (all_symp_col)]

In [10]:
## get all symptoms which do not have a synset
ohne_syns=[]
mit_syns=[]
for sym in all_symp:
    if not wn.synsets(sym) :
        ohne_syns.append(sym)
    else:
        mit_syns.append(sym)

In [11]:
len(ohne_syns)

99

In [12]:
mit_syns

['itching',
 'shivering',
 'chills',
 'acidity',
 'vomiting',
 'fatigue',
 'anxiety',
 'restlessness',
 'lethargy',
 'cough',
 'breathlessness',
 'sweating',
 'dehydration',
 'indigestion',
 'headache',
 'nausea',
 'constipation',
 'diarrhoea',
 'malaise',
 'phlegm',
 'congestion',
 'dizziness',
 'cramps',
 'bruising',
 'obesity',
 'unsteadiness',
 'depression',
 'irritability',
 'polyuria',
 'coma',
 'palpitations',
 'blackheads',
 'blister']

In [13]:
ohne_syns

['skin rash',
 'nodal skin eruptions',
 'continuous sneezing',
 'joint pain',
 'stomach pain',
 'ulcers on tongue',
 'muscle wasting',
 'burning micturition',
 'spotting  urination',
 'weight gain',
 'cold hands and feets',
 'mood swings',
 'weight loss',
 'patches in throat',
 'irregular sugar level',
 'high fever',
 'sunken eyes',
 'yellow skin',
 'dark urine',
 'loss of appetite',
 'pain behind the eyes',
 'back pain',
 'abdominal pain',
 'mild fever',
 'yellow urine',
 'yellow of eyes',
 'acute liver failure',
 'fluid overload',
 'swelling of stomach',
 'swelled lymph nodes',
 'blurred and distorted vision',
 'throat irritation',
 'redness of eyes',
 'sinus pressure',
 'runny nose',
 'chest pain',
 'weakness in limbs',
 'fast heart rate',
 'pain during bowel movements',
 'pain in anal region',
 'bloody stool',
 'irritation in anus',
 'neck pain',
 'swollen legs',
 'swollen blood vessels',
 'puffy face and eyes',
 'enlarged thyroid',
 'brittle nails',
 'swollen extremeties',
 'exces

# II- Preprocess text

In [14]:
from spacy.lang.en.stop_words import STOP_WORDS
import spacy
nlp = spacy.load('en_core_web_sm')

In [89]:
def preprocess(doc):
    nlp_doc=nlp(doc)
    d=[]
    for token in nlp_doc:
        if(not token.text.lower()  in STOP_WORDS and  token.text.isalpha() and token.tag_ in ("JJ","NN","VBG","NNS")):
            d.append(token.lemma_.lower() )
    return ' '.join(d)

In [16]:
def preprocess_sym(doc):
    nlp_doc=nlp(doc)
    d=[]
    for token in nlp_doc:
        if(not token.text.lower()  in STOP_WORDS and  token.text.isalpha()):
            d.append(token.lemma_.lower() )
    return ' '.join(d)

In [90]:
preprocess("skin peeling")

'skin peel'

In [18]:
all_symp_pr=[preprocess_sym(sym) for sym in all_symp]

In [19]:
#associe chaque symp pretraite au non de sa colonne originale
col_dict = dict(zip(all_symp_pr, all_symp_col))

# III- Syntactic Similarity

In [20]:
def jaccard_set(str1, str2):
    list1=str1.split(' ')
    list2=str2.split(' ')
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [21]:
#similarite syn avec ts le corpus
def syntactic_similarity(symp_t,corpus):
    max_sim=0
    most_sim=None
    for symp in corpus:
        d=jaccard_set(symp_t,symp)
        if d>max_sim:
            most_sim=symp
            max_sim=d
    return max_sim,most_sim



In [22]:
syntactic_similarity('nail',all_symp_pr)

(0.5, 'brittle nail')

In [23]:
def check_pattern(inp,dis_list):
    import re
    pred_list=[]
    ptr=0
    patt = "^" + inp + "$"
    regexp = re.compile(inp)
    for item in dis_list:
        if regexp.search(item):
            pred_list.append(item)
    if(len(pred_list)>0):
        return 1,pred_list
    else:
        return ptr,None

In [24]:
check_pattern('rwa',all_symp_pr)

(0, None)

# IV- Semantic Similarity

In [25]:
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize
def WSD(word, context):
    sens=lesk(context, word)
    return sens

In [26]:
def semanticD(doc1,doc2):
    doc1_p=preprocess(doc1).split(' ')
    doc2_p=preprocess_sym(doc2).split(' ')
    score=0
    for tock1 in doc1_p:
        for tock2 in doc2_p:
            syn1 = WSD(tock1,doc1)
            syn2 = WSD(tock2,doc2)
            if syn1 is not None and syn2 is not None :
                x=syn1.path_similarity(syn2)
                if x is not None and x>0.25:
                    score+=x
    return score/(len(doc1_p)*len(doc2_p))

In [27]:
#similarite sem avec ts le corpus
def semantic_similarity(symp_t,corpus):
    max_sim=0
    most_sim=None
    for symp in corpus:
        d=semanticD(symp_t,symp)
        if d>max_sim:
            most_sim=symp
            max_sim=d
    return max_sim,most_sim

In [28]:
semantic_similarity('puke',all_symp_pr)

(1.0, 'vomit')

In [29]:
from itertools import chain
from nltk.corpus import wordnet
def suggest_syn(sym):
    symp=[]
    synonyms = wordnet.synsets(sym)
    lemmas=[word.lemma_names() for word in synonyms]
    lemmas = list(set(chain(*lemmas)))
    for e in lemmas:
        res,sym1=semantic_similarity(e,all_symp_pr)
        if res!=0:
            symp.append(sym1)
    return list(set(symp))

In [30]:
suggest_syn('puke')

['pain', 'vomit', 'dark urine']

In [31]:
#recoit client_symptoms et renvoit un dataframe avec 1 pour les symptoms associees
def OHV(cl_sym,all_sym):
    l=np.zeros([1,len(all_sym)])
    for sym in cl_sym:
        l[0,all_sym.index(sym)]=1
    return pd.DataFrame(l, columns =all_symp)
    

In [32]:
def contains(small, big):
    a=True
    for i in small:
        if i not in big:
            a=False
    return a
        

In [33]:
def possible_diseases(l):
    poss_dis=[]
    for dis in set(disease):
        if contains(l,symVONdisease(df_tr,dis)):
            poss_dis.append(dis)
    return poss_dis

In [34]:
#recoit une maladie renvoit tous les sympts
def symVONdisease(df,disease):
    ddf=df[df.prognosis==disease]
    m2 = (ddf == 1).any()
    return m2.index[m2].tolist()
    

In [35]:
def main(sym1,sym2,all_symp_col):
    #1 st text
    sym1=preprocess(sym1)
    sym2=preprocess(sym2)
    sim1,psym1=syntactic_similarity(sym1,all_symp_pr)
    sim2,psym2=syntactic_similarity(sym2,all_symp_pr)
    if sim1<0.6:
        sim1,psym1=semantic_similarity(sym1,all_symp_pr)
        if sim1==0:
            sugg=suggest_syn(sym1)
            for res in sugg:
                inp=input('Do you feel '+ res+" ?(yes or no) ")
                if inp=="yes":
                    psym1=res
                    sim1=1
                    break
    if sim2<0.6:
        sim2,psym2=semantic_similarity(sym2,all_symp_pr)
        if sim2==0:
            sugg=suggest_syn(sym2)
            for res in sugg:
                inp=input('Do you feel '+ res+" ?(yes or no) ")
                if inp=="yes":
                    psym2=res
                    sim2=1
                    break

    if sim1==0 and sim2==0:
        return None
    else:
        if sim1==0:
            psym1=psym2
        if sim2==0:
            psym2=psym1
        all_sym=[col_dict[psym1],col_dict[psym2]]
        diseases=possible_diseases(all_sym)
        stop=False
        for dis in diseases:
            if stop==False:
                for sym in symVONdisease(df_tr,dis):
                    if sym not in all_sym:
                        inp=input('Do you feel '+ sym +" ?(yes or no) ")
                        if inp=="yes":
                            all_sym.append(sym)
                            diseases=possible_diseases(all_sym)
                            if len(diseases)==1:
                                stop=True 
                                break

                        else:
                            continue
        return knn_clf.predict(OHV(all_sym,all_symp_col))

    
    

In [36]:
symVONdisease(df_tr,'Drug Reaction')

['itching',
 'skin_rash',
 'stomach_pain',
 'burning_micturition',
 'spotting_ urination']

In [37]:
main('skin rash','itching',all_symp_col)

Do you feel fatigue ?(yes or no) h


KeyboardInterrupt: 

# V- Prediction Model (KNN & DT)

In [35]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [36]:
X_train=df_tr.iloc[:,:-1]
X_test=df_tt.iloc[:,:-1]
y_train = df_tr.iloc[:,-1]
y_test = df_tt.iloc[:,-1]

In [37]:
knn_clf=KNeighborsClassifier()
knn_clf.fit(X_train,y_train)

KNeighborsClassifier()

In [38]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [39]:
print(classification_report(y_test,knn_clf.predict(X_test)))

                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00         1
                                   AIDS       1.00      1.00      1.00         1
                                   Acne       1.00      1.00      1.00         1
                    Alcoholic hepatitis       1.00      1.00      1.00         1
                                Allergy       1.00      1.00      1.00         1
                              Arthritis       1.00      1.00      1.00         1
                       Bronchial Asthma       1.00      1.00      1.00         1
                   Cervical spondylosis       1.00      1.00      1.00         1
                            Chicken pox       1.00      1.00      1.00         1
                    Chronic cholestasis       1.00      1.00      1.00         1
                            Common Cold       1.00      1.00      1.00         1
                           

In [40]:
print(classification_report(y_test,dt_clf.predict(X_test)))

                                         precision    recall  f1-score   support

(vertigo) Paroymsal  Positional Vertigo       1.00      1.00      1.00         1
                                   AIDS       1.00      1.00      1.00         1
                                   Acne       1.00      1.00      1.00         1
                    Alcoholic hepatitis       1.00      1.00      1.00         1
                                Allergy       1.00      1.00      1.00         1
                              Arthritis       1.00      1.00      1.00         1
                       Bronchial Asthma       1.00      1.00      1.00         1
                   Cervical spondylosis       1.00      1.00      1.00         1
                            Chicken pox       1.00      1.00      1.00         1
                    Chronic cholestasis       1.00      1.00      1.00         1
                            Common Cold       1.00      1.00      1.00         1
                           

# Chat

In [53]:
def getInfo():
    # name=input("Name:")
    print("Your Name \n\t\t\t\t\t\t",end="=>")
    name=input("")
    print("hello ",name)

In [45]:
def chat():
    a=True
    while(a):
        ans=input("What do you feel ?")
        ans2=input("What do you feel more ?")
        result=main(ans,ans2,all_symp_col)
        if result == None :
            ans3=input("can you specify more what you feel or tap q to stop the conversation")
            if ans3=="q":
                a=False
            else:
                continue
            
        else:
            print("you may have "+result[0])
            print("do you need another medical consultation (yes or no)? ")
            ans=input()
            if ans!="yes":
                a=False
                print("!!!!! thanks for using ower application !!!!!! ")


In [57]:
def related_sym(psym1):
    print("searches related to input: ")
    for num,it in enumerate(psym1):
        print(num,")",it)
    if num!=0:
        print(f"Select the one you meant (0 - {num}):  ", end="")
        conf_inp = int(input(""))
    else:
        conf_inp=0

    disease_input=psym1[conf_inp]
    return disease_input

In [99]:
def main_sp(sym1,sym2,all_symp_col):
    sym1=preprocess_sym(sym1)
    sym2=preprocess_sym(sym2)
    sim1,psym1=check_pattern(sym1,all_symp_pr)
    if sim1==1 :
        psym1=related_sym(psym1)
    sim2,psym2=check_pattern(sym2,all_symp_pr)
    if sim2==1 :
        psym2=related_sym(psym2)
    if sim1==0 or sim2==0:
        
        sim1,psym1=semantic_similarity(sym1,all_symp_pr)
        sim2,psym2=semantic_similarity(sym2,all_symp_pr)
        if sim1==0:
            sugg=suggest_syn(sym1)
            for res in sugg:
                inp=input('Do you feel '+ res+" ?(yes or no) ")
                if inp=="yes":
                    psym1=res
                    sim1=1
                    break
        if sim2==0:
            sugg=suggest_syn(sym2)
            for res in sugg:
                inp=input('Do you feel '+ res+" ?(yes or no) ")
                if inp=="yes":
                    psym2=res
                    sim2=1
                    break

        if sim1==0 and sim2==0:
            return None,None
        else:
            if sim1==0:
                psym1=psym2
            if sim2==0:
                psym2=psym1
    print(psym1)
    print(psym2)
    all_sym=[col_dict[psym1],col_dict[psym2]]
    diseases=possible_diseases(all_sym)
    print(diseases)
    stop=False
    for dis in diseases:
        if stop==False:
            for sym in symVONdisease(df_tr,dis):
                if sym not in all_sym:
                    inp=input('Do you feel '+ sym +" ?(yes or no) ")
                    if inp=="yes":
                        all_sym.append(sym)
                        dise=possible_diseases(all_sym)
                        if len(dise)==1:
                            stop=True 
                            break
                    else:
                        continue
    return knn_clf.predict(OHV(all_sym,all_symp_col)),all_sym

    
    

In [95]:
def chat_sp():
    a=True
    while a:
        getInfo()

        print("Enter the main symptom you are experiencing  \n\t\t\t\t\t\t",end="=>")
        ans = input("")
        print("Enter a second symptom you are experiencing  \n\t\t\t\t\t\t",end="=>")
        ans2=input("")
        result,sym=main_sp(ans,ans2,all_symp_col)
        if result == None :
            ans3=input("can you specify more what you feel or tap q to stop the conversation")
            if ans3=="q":
                a=False
            else:
                continue

        else:
            print("you may have "+result[0])
            print(description_list[result[0]])
            an=input("how many day do you feel those symptoms ?")
            if calc_condition(sym,int(an))==1:
                print("you should take the consultation from doctor")
            else : 
                print('Take following precautions : ')
                for e in precautionDictionary[result[0]]:
                    print(e)
            print("do you need another medical consultation (yes or no)? ")
            ans=input()
            if ans!="yes":
                a=False
                print("!!!!! thanks for using ower application !!!!!! ")


In [97]:
chat_sp()

Your Name 
						=>itching
hello  itching
Enter the main symptom you are experiencing  
						=>itching
Enter a second symptom you are experiencing  
						=>fever
searches related to input: 
0 ) itch
1 ) internal itching
Select the one you meant (0 - 1):  0
searches related to input: 
0 ) high fever
1 ) mild fever
Select the one you meant (0 - 1):  0
itch
high fever
['Jaundice', 'Chicken pox']
Do you feel vomiting ?(yes or no) yes
Do you feel fatigue ?(yes or no) yes
Do you feel weight_loss ?(yes or no) yes
Do you feel yellowish_skin ?(yes or no) no
Do you feel dark_urine ?(yes or no) no
Do you feel abdominal_pain ?(yes or no) yes
Do you feel skin_rash ?(yes or no) yes
Do you feel lethargy ?(yes or no) yes
Do you feel headache ?(yes or no) yes
Do you feel loss_of_appetite ?(yes or no) yes
Do you feel mild_fever ?(yes or no) yes
Do you feel swelled_lymph_nodes ?(yes or no) yes
Do you feel malaise ?(yes or no) yes
Do you feel red_spots_over_body ?(yes or no) yes
you may have Chicken pox

In [74]:
possible_diseases([col_dict["blister"],col_dict["skin rash"]])

['Impetigo']

In [82]:
symVONdisease(df_tr,'Hepatitis B')

['itching',
 'fatigue',
 'lethargy',
 'yellowish_skin',
 'dark_urine',
 'loss_of_appetite',
 'abdominal_pain',
 'yellow_urine',
 'yellowing_of_eyes',
 'malaise',
 'receiving_blood_transfusion',
 'receiving_unsterile_injections']

##  VI- SEVERITY / DESCRIPTION / PRECAUTION

In [42]:

severityDictionary=dict()
description_list = dict()
precautionDictionary=dict()

def getDescription():
    global description_list
    with open('symptom_Description.csv') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _description={row[0]:row[1]}
            description_list.update(_description)




def getSeverityDict():
    global severityDictionary
    with open('symptom_severity.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        try:
            for row in csv_reader:
                _diction={row[0]:int(row[1])}
                severityDictionary.update(_diction)
        except:
            pass


def getprecautionDict():
    global precautionDictionary
    with open('symptom_precaution.csv') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            _prec={row[0]:[row[1],row[2],row[3],row[4]]}
            precautionDictionary.update(_prec)


In [43]:
getSeverityDict()
getprecautionDict()
getDescription()

In [49]:
def calc_condition(exp,days):
    sum=0
    for item in exp:
         sum=sum+severityDictionary[item]
    if((sum*days)/(len(exp))>13):
        return 1
        print("You should take the consultation from doctor. ")
    else:
        return 0
        print("It might not be that bad but you should take precautions.")


In [84]:
calc_condition(['itching','skin_rash'],8)

You should take the consultation from doctor. 


In [91]:
description_list['Drug Reaction']

'An adverse drug reaction (ADR) is an injury caused by taking medication. ADRs may occur following a single dose or prolonged administration of a drug or result from the combination of two or more drugs.'

In [93]:
precautionDictionary['Drug Reaction']

['stop irritation',
 'consult nearest hospital',
 'stop taking drug',
 'follow up']